In [1]:
#We will implement 2 layer Neural Network

In [2]:
import numpy as np

In [3]:
X = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([[0],[1],[1],[0]])

In [4]:
X = X.reshape(X.shape[0], -1).T
X

array([[0, 0, 1, 1],
       [0, 1, 0, 1]])

In [5]:
def sigmoid(Z):
    A =  1 / (1 + np.exp(-Z))
    cache = Z
    return A,cache

def back_sigmoid(dA,cache):
    Z = cache
    s = 1 / (1 + np.exp(-Z))
    dZ = dA * s * (1-s)
    assert (dZ.shape == Z.shape)
    return dZ

In [6]:
def relu(Z):
    A = np.maximum(0,Z)
    assert(A.shape == Z.shape)
    cache = Z
    return A,cache

def back_relu(dA,cache):
    Z = cache
    dZ = np.array(dA, copy=True)
    dZ[Z<= 0] = 0
    assert(dZ.shape == Z.shape)
    return dZ


In [7]:
def init(n_x,n_h,n_y):
    W1 = np.random.randn(n_h,n_x)*0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y,n_h)*0.01
    b2 = np.zeros((n_y,1))
    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))
    parameters = {
        "W1":W1,
        "b1":b1,
        "W2":W2,
        "b2":b2,
    }
    return parameters

In [8]:
def linear_forward(A,W,b):
    Z = np.dot(W,A) + b
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A,W,b)
    return Z ,cache

In [9]:
def linear_forward_activation(A_prev,W,b,activation):
    if(activation == "relu"):
        Z,linear_cahe = linear_forward(A_prev,W,b)
        A,activation_cache = relu(Z)
    elif activation == "sigmoid":
        Z,linear_cahe = linear_forward(A_prev,W,b)
        A,activation_cache = sigmoid(Z)
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cahe,activation_cache)
    return A,cache

In [10]:
def compute_cost(AL,Y):
    m = Y.shape[1]
    cost = (1./m) * (-np.dot(Y,np.log(AL).T) - np.dot(1-Y, np.log(1-AL).T))
    cost = np.squeeze(cost) 
    assert(cost.shape == ())
    return cost

In [11]:
def linear_backward(dZ,cache):
    A_prev,W,b = cache
    m = A_prev.shape[1]
    dW = 1./m * np.dot(dZ,A_prev.T)
    db = 1./m * np.sum(dZ,axis= 1 ,keepdims= True)
    dA_prev = np.dot(W.T,dZ)

    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    return dA_prev,dW,db

In [12]:
def linear_activation_back(dA,cache,activation):
    linear_cache,activation_cache = cache
    if activation == "relu":
        dZ = back_relu(dA,activation_cache)
        dA_prev,dW,db = linear_backward(dZ,linear_cache)
    elif activation == "sigmoid":
        dZ = back_sigmoid(dA,activation_cache)
        dA_prev,dW,db = linear_backward(dZ,linear_cache)
    return dA_prev,dW,db

In [13]:
def update_parameter(parameters,grads,learning_rate):
    parameters["W1"] = parameters['W1'] - learning_rate*grads["dW1"]
    parameters["b1"] = parameters['b1'] - learning_rate*grads["db1"]
    parameters["W2"] = parameters['W2'] - learning_rate*grads["dW2"]
    parameters["b2"] = parameters['b2'] - learning_rate*grads["db2"]
    return parameters

In [14]:
def model(X,Y,layer_dims,learning_rate = 0.01,num_iterations = 3000):
    grads = {}
    costs = []
    m = X.shape[1]
    print(m)
    (n_x,n_h,n_y) = layer_dims
    parameters = init(n_x,n_h,n_y)

    W1 = parameters['W1']
    b2 = parameters['b2']
    W2 = parameters['W2']
    b1 = parameters['b1']

    for i in range(num_iterations):
        A1,cache = linear_forward_activation(X,W1.T,b1,"relu")
        A2,cache = linear_forward_activation(A1,W2.T,b2,"sigmoid")
        cost = compute_cost(A2,Y)
       
        dA2 = - (np.divide(Y,A2) - np.divide(1-Y,1-A2))

        dA1,dW2,db2 = linear_activation_back(dA2,cache,"sigmoid")
        print(f'its da1 {dA1}')
        dA0,dW1,db1 = linear_activation_back(dA1,cache,"relu")
       
        grads['dW1'] = dW1
        grads['db1'] = db1
        grads['dW2'] = dW2
        grads['db2'] = db2

        parameters = update_parameter(parameters,grads,learning_rate)

        W1 = parameters["W1"]
        b1 = parameters["b1"]
        W2 = parameters["W2"]
        b2 = parameters["b2"]

        if i % 100 == 0:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0:
            costs.append(cost)

    return parameters



In [15]:
def predict(X, y, parameters):
    m = X.shape[1]
    n = len(parameters) // 2 # number of layers in the neural network
    p = np.zeros((1,m))
    
    # Forward propagation
    probas, caches = linear_forward_activation(X, parameters)

    
    # convert probas to 0/1 predictions
    for i in range(0, probas.shape[1]):
        if probas[0,i] > 0.5:
            p[0,i] = 1
        else:
            p[0,i] = 0
    print("Accuracy: "  + str(np.sum((p == y)/m)))
        
    return p

In [21]:
paramertes = model(X[0],y,[1,4,1],0.01,1000)

IndexError: tuple index out of range